In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pandas as pd
import numpy as np

# try:
#     import google.colab
#     COLAB = True
#     print("Note: using Google CoLab")
# except:
#     print("Note: not using Google CoLab")
#     COLAB = False

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
# import torch
# has_mps = torch.backends.mps.is_built()
# device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
# print(f"Using device: {device}")

device = "cuda" if torch.cuda.is_available() else "cpu"

## Preprocess data

In [2]:
df_train = pd.read_csv('/mnt/4E280AEC280AD33F/Projects/CU/CUEE_TimeSeries_Forcasting/solar-LTFS/dataset/CUEE_PMAPS/pmaps_train_data.csv', parse_dates=['Datetime'])
df_val  = pd.read_csv('/mnt/4E280AEC280AD33F/Projects/CU/CUEE_TimeSeries_Forcasting/solar-LTFS/dataset/CUEE_PMAPS/pmaps_validate_data.csv', parse_dates=['Datetime'])
df_test = pd.read_csv('/mnt/4E280AEC280AD33F/Projects/CU/CUEE_TimeSeries_Forcasting/solar-LTFS/dataset/CUEE_PMAPS/pmaps_test_data.csv', parse_dates=['Datetime'])

In [3]:
df_train.iloc[:40] 


,Datetime,site_name,I,Iclr,latt,long,CI,R,hour_encode1,day,month,minute,temperature,I_nwp
0,2022-04-01 07:30:00+07:00,ISL001,235.5784,265.993700,14.00523,100.519403,14.0,78.0,-4.5,91,4,30,28.49,158.9992
1,2022-04-01 07:45:00+07:00,ISL001,322.4936,334.819476,14.00523,100.519403,14.0,78.0,-4.5,91,4,45,28.79,201.5592
2,2022-04-01 08:00:00+07:00,ISL001,357.6841,402.581345,14.00523,100.519403,13.0,74.0,-3.5,91,4,0,29.08,245.8656
3,2022-04-01 08:15:00+07:00,ISL001,413.5059,468.732095,14.00523,100.519403,13.0,74.0,-3.5,91,4,15,29.38,291.6676
4,2022-04-01 08:30:00+07:00,ISL001,471.1933,532.843024,14.00523,100.519403,13.0,74.0,-3.5,91,4,30,29.67,338.6980
5,2022-04-01 08:45:00+07:00,ISL001,536.3066,594.551229,14.00523,100.519403,13.0,74.0,-3.5,91,4,45,29.97,386.4820
6,2022-04-01 09:00:00+07:00,ISL001,604.1268,653.535674,14.00523,100.519403,13.0,74.0,-2.5,91,4,0,30.26,434.6692
7,2022-04-01 09:15:00+07:00,ISL001,661.4119,709.505601,14.00523,100.519403,13.0,74.0,-2.5,91,4,15,30.56,483.1280
8,2022-04-01 09:30:00+07:00,ISL001,723.4158,762.194726,14.00523,100.519403,17.0,77.0,-2.5,91,4,30,30.85,531.5432
9,2022-04-01 09:45:00+07:00,ISL001,773.8237,811.358317,14.00523,100.519403,17.0,77.0,-2.5,91,4,45,31.15,579.5912


In [4]:
# Prepare the training data
train_data = df_train[['Iclr', 'CI', 'R', 'hour_encode1', 'day', 'month', 'minute', 'latt', 'long', 'temperature', 'I_nwp']]
train_target = df_train['I']

# Scale the training data
scaler = StandardScaler()
train_data_scaled = scaler.fit_transform(train_data)

# Convert training data to tensors
Iclrs_train = torch.tensor(train_data_scaled[:, 0], dtype=torch.float32)
CIs_train = torch.tensor(train_data_scaled[:, 1], dtype=torch.float32)
Rs_train = torch.tensor(train_data_scaled[:, 2], dtype=torch.float32)
hour_encode1s_train = torch.tensor(train_data_scaled[:, 3], dtype=torch.float32)
days_train = torch.tensor(train_data_scaled[:, 4], dtype=torch.float32)
months_train = torch.tensor(train_data_scaled[:, 5], dtype=torch.float32)
minute_train = torch.tensor(train_data_scaled[:, 6], dtype=torch.float32)
latts_train = torch.tensor(train_data_scaled[:, 7], dtype=torch.float32)
longs_train = torch.tensor(train_data_scaled[:, 8], dtype=torch.float32)
temps_train = torch.tensor(train_data_scaled[:, 9], dtype=torch.float32)
I_nwps_train = torch.tensor(train_data_scaled[:, 10], dtype=torch.float32)


# Scale the training target
scaler_target = StandardScaler()
train_target_scaled = scaler_target.fit_transform(train_target.values.reshape(-1, 1))

# Convert training target to tensors
Is_train = torch.tensor(train_target_scaled, dtype=torch.float32)

# Prepare the validate data
val_data = df_val[['Iclr', 'CI', 'R', 'hour_encode1', 'day', 'month', 'minute', 'latt', 'long', 'temperature', 'I_nwp']]
val_target = df_val['I']

# Scale the validate data using the scaler fitted on the training data
val_data_scaled = scaler.transform(val_data)

# Convert validate data to tensors
Iclrs_val = torch.tensor(val_data_scaled[:, 0], dtype=torch.float32)
CIs_val = torch.tensor(val_data_scaled[:, 1], dtype=torch.float32)
Rs_val = torch.tensor(val_data_scaled[:, 2], dtype=torch.float32)
hour_encode1s_val = torch.tensor(val_data_scaled[:, 3], dtype=torch.float32)
days_val = torch.tensor(val_data_scaled[:, 4], dtype=torch.float32)
months_val = torch.tensor(val_data_scaled[:, 5], dtype=torch.float32)
minute_val = torch.tensor(val_data_scaled[:, 6], dtype=torch.float32)
latts_val = torch.tensor(val_data_scaled[:, 7], dtype=torch.float32)
longs_val = torch.tensor(val_data_scaled[:, 8], dtype=torch.float32)
temps_val = torch.tensor(val_data_scaled[:, 9], dtype=torch.float32)
I_nwps_val = torch.tensor(val_data_scaled[:, 10], dtype=torch.float32)


# Scale the validate target using the scaler fitted on the training target
val_target_scaled = scaler_target.transform(val_target.values.reshape(-1, 1))

# Convert validate target to tensors
Is_val = torch.tensor(val_target_scaled, dtype=torch.float32)

In [5]:
# Sequence Data Preparation
SEQUENCE_SIZE = 4

def to_sequences(seq_size, obs_Iclrs, obs_CIs, obs_Rs, obs_hour_encode1s, obs_days, obs_months, obs_minutes, obs_Iwnp, obs_latts, obs_longs, obs_temp ,obs_Is):
    x_data = []
    y_data = []
    for i in range(len(obs_Iclrs) - seq_size):
        window_Iclrs = obs_Iclrs[i:(i + seq_size)]
        window_CIs = obs_CIs[i:(i + seq_size)]
        window_Rs = obs_Rs[i:(i + seq_size)]
        window_hour_encode1s = obs_hour_encode1s[i:(i + seq_size)]
        window_days = obs_days[i:(i + seq_size)]
        window_months = obs_months[i:(i + seq_size)]
        window_minutes = obs_minutes[i:(i + seq_size)]
        window_Iwnp = obs_Iwnp[i:(i + seq_size)]
        window_temp = obs_temp[i:(i + seq_size)]

        # lag term are repeated
        window_latts = obs_latts[i:i+1].repeat(seq_size)
        window_longs = obs_longs[i:i+1].repeat(seq_size)
        
        after_window = obs_Is[i + seq_size]
        x_window = torch.stack((window_Iclrs, window_CIs, window_Rs, window_hour_encode1s, window_days, window_months, window_minutes, window_Iwnp, 
                                window_latts,  window_longs, window_temp), dim=1)
        x_data.append(x_window) 
        y_data.append(after_window)
    x = torch.stack(x_data)
    y = torch.stack(y_data)
    return x, y

x_train, y_train = to_sequences(SEQUENCE_SIZE, Iclrs_train, CIs_train, Rs_train, hour_encode1s_train, days_train, months_train, minute_train, I_nwps_train,
                                latts_train, longs_train, temps_train,  Is_train)
x_val, y_val = to_sequences(SEQUENCE_SIZE, Iclrs_val, CIs_val, Rs_val, hour_encode1s_val, days_val, months_val, minute_val, I_nwps_val, 
                            latts_val, longs_val, temps_val, Is_val)

In [6]:
x_train.shape, y_train.shape, x_val.shape, y_val.shape

(torch.Size([416608, 4, 11]),
 torch.Size([416608, 1]),
 torch.Size([104073, 4, 11]),
 torch.Size([104073, 1]))

In [7]:
# Setup data loaders for batch --> check shuffle = True
train_dataset = TensorDataset(x_train, y_train)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = TensorDataset(x_val, y_val)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)

## Train model

In [8]:
import torch.nn as nn

# Define the LSTM model

class LSTMModel(nn.Module):
    def __init__(self):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size=11, hidden_size=50, batch_first=True)
        self.activation = nn.ReLU()
        self.dropout = nn.Dropout(0.1)
        self.fc = nn.Linear(50, 1)

    def forward(self, x):
        x, _ = self.lstm(x)
        x = self.activation(x)
        x = self.dropout(x)
        x = self.fc(x[:, -1, :])
        return x
model = LSTMModel().to(device)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 12,651 trainable parameters


In [9]:
# Train the model
criterion = nn.L1Loss()  # Mean Absolute Error (MAE) loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=3, verbose=True)

epochs = 100
early_stop_count = 0
min_val_loss = float('inf')

for epoch in range(epochs):

    # Training
    model.train()
    train_losses = []
    for batch in train_loader:
        x_batch, y_batch = batch
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)
        optimizer.zero_grad()
        outputs = model(x_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_losses.append(loss.item())
    
    train_loss = np.mean(train_losses)
    
    # Validation
    model.eval()
    val_losses = []
    with torch.no_grad():
        for batch in val_loader:
            x_batch, y_batch = batch
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            outputs = model(x_batch)
            loss = criterion(outputs, y_batch)
            val_losses.append(loss.item())
    
    val_loss = np.mean(val_losses)
    scheduler.step(val_loss)
    
    if val_loss < min_val_loss:
        min_val_loss = val_loss
        early_stop_count = 0
    else:
        early_stop_count += 1
    
    if early_stop_count >= 5:
        print("Early stopping!")
        break
    
    print(f"Epoch {epoch + 1}/{epochs}, Train Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

/home/gapp/anaconda3/envs/Torch312/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "


Epoch 1/100, Train Loss: 0.3368, Validation Loss: 0.3189
Epoch 2/100, Train Loss: 0.3260, Validation Loss: 0.3163
Epoch 3/100, Train Loss: 0.3227, Validation Loss: 0.3132
Epoch 4/100, Train Loss: 0.3203, Validation Loss: 0.3095
Epoch 5/100, Train Loss: 0.3187, Validation Loss: 0.3095
Epoch 6/100, Train Loss: 0.3168, Validation Loss: 0.3084
Epoch 7/100, Train Loss: 0.3155, Validation Loss: 0.3073
Epoch 8/100, Train Loss: 0.3145, Validation Loss: 0.3057
Epoch 9/100, Train Loss: 0.3130, Validation Loss: 0.3053
Epoch 10/100, Train Loss: 0.3122, Validation Loss: 0.3041
Epoch 11/100, Train Loss: 0.3114, Validation Loss: 0.3035
Epoch 12/100, Train Loss: 0.3105, Validation Loss: 0.3049
Epoch 13/100, Train Loss: 0.3097, Validation Loss: 0.3028
Epoch 14/100, Train Loss: 0.3089, Validation Loss: 0.3030
Epoch 15/100, Train Loss: 0.3084, Validation Loss: 0.3047
Epoch 16/100, Train Loss: 0.3078, Validation Loss: 0.3028
Epoch 17/100, Train Loss: 0.3071, Validation Loss: 0.3035
Epoch 18/100, Train Los

## Evaluate the model in the test set

In [10]:
df_test

,Datetime,site_name,I,Iclr,latt,long,CI,R,hour_encode1,day,month,minute,temperature,I_nwp
0,2022-04-05 07:00:00+07:00,ISL001,183.5975,141.241660,14.00523,100.519403,23.0,52.0,-4.5,95,4,0,25.04,114.2596
1,2022-04-05 07:15:00+07:00,ISL001,240.5533,209.518060,14.00523,100.519403,23.0,52.0,-4.5,95,4,15,25.56,167.5216
2,2022-04-05 07:30:00+07:00,ISL001,164.2464,278.611744,14.00523,100.519403,36.0,76.0,-4.5,95,4,30,26.07,222.8948
3,2022-04-05 07:45:00+07:00,ISL001,176.0619,347.292820,14.00523,100.519403,36.0,76.0,-4.5,95,4,45,26.58,281.1156
4,2022-04-05 08:00:00+07:00,ISL001,261.4042,414.830234,14.00523,100.519403,44.0,147.0,-3.5,95,4,0,27.10,341.8636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
347118,2023-06-22 16:00:00+07:00,ISL056,473.4280,660.558000,13.50130,100.135400,6.0,72.0,4.5,173,6,0,33.33,563.5720
347119,2023-06-22 16:15:00+07:00,ISL056,418.3324,603.006077,13.50130,100.135400,6.0,72.0,4.5,173,6,15,32.94,513.7404
347120,2023-06-22 16:30:00+07:00,ISL056,367.2727,543.148095,13.50130,100.135400,6.0,60.0,4.5,173,6,30,32.56,463.2660
347121,2023-06-22 16:45:00+07:00,ISL056,311.9268,481.265865,13.50130,100.135400,6.0,60.0,4.5,173,6,45,32.18,412.4092


In [11]:
# Prepare the testing data
test_data = df_test[['Iclr', 'CI', 'R', 'hour_encode1', 'day', 'month', 'minute', 'latt', 'long', 'temperature', 'I_nwp']]
test_target = df_test['I']

# Scale the testing data using the scaler fitted on the training data
test_data_scaled = scaler.transform(test_data)

# Scale the testing target using the scaler fitted on the training target
test_target_scaled = scaler_target.transform(test_target.values.reshape(-1, 1))

# Convert testing data to tensors
Iclrs_test = torch.tensor(test_data_scaled[:, 0], dtype=torch.float32)
CIs_test = torch.tensor(test_data_scaled[:, 1], dtype=torch.float32)
Rs_test = torch.tensor(test_data_scaled[:, 2], dtype=torch.float32)
hour_encode1s_test = torch.tensor(test_data_scaled[:, 3], dtype=torch.float32)
days_test = torch.tensor(test_data_scaled[:, 4], dtype=torch.float32)
months_test = torch.tensor(test_data_scaled[:, 5], dtype=torch.float32)
minute_test = torch.tensor(test_data_scaled[:, 6], dtype=torch.float32)
latts_test = torch.tensor(test_data_scaled[:, 7], dtype=torch.float32)
longs_test = torch.tensor(test_data_scaled[:, 8], dtype=torch.float32)
temps_test = torch.tensor(test_data_scaled[:, 9], dtype=torch.float32)
I_nwps_test = torch.tensor(test_data_scaled[:, 10], dtype=torch.float32)

# Convert testing target to tensors
Is_test = torch.tensor(test_target_scaled, dtype=torch.float32)

In [12]:
x_test, y_test = to_sequences(SEQUENCE_SIZE, Iclrs_test, CIs_test, Rs_test, hour_encode1s_test, days_test, months_test, minute_test, I_nwps_test,
                              latts_test, longs_test, temps_test, Is_test)

test_dataset = TensorDataset(x_test, y_test)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [13]:
# Evaluation
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_loader:
        x_batch, y_batch = batch
        x_batch = x_batch.to(device)
        outputs = model(x_batch)
        predictions.extend(outputs.cpu().squeeze().tolist())

predictions = np.array(predictions).reshape(-1, 1)
y_test_numpy = y_test.numpy().reshape(-1, 1)

# Inverse transform the predictions and y_test using the scaler_target
predictions_inverse = scaler_target.inverse_transform(predictions)
y_test_inverse = scaler_target.inverse_transform(y_test_numpy)

# Create dataframe
df_results = pd.DataFrame({'I': y_test_inverse.flatten(), 'Ihat': predictions_inverse.flatten()})

# Calculate the RMSE
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = np.sqrt(mean_squared_error(df_results['I'], df_results['Ihat']))
mae = mean_absolute_error(df_results['I'], df_results['Ihat'])
print(f"RMSE: {rmse}", f"MAE: {mae}")

RMSE: 130.24457116939465 MAE: 83.90694733861714


In [ ]:
# prepare the results dataframe for calculating metrics in different sky conditions and times

df_results['datetime'] = df_test['Datetime'].iloc[:-1]
df_results['datetime'] = pd.to_datetime(df_results['datetime'])
df_results['site_name'] = df_test['site_name'].iloc[:-1]
df_results['Iclr'] = df_test['Iclr'].iloc[:-1]
df_results['k'] = df_results['I'] / df_results['Iclr']
df_results['Date'] = df_results['datetime'].dt.date
df_results['hour'] = df_results['datetime'].dt.hour

# create K_bar in each day and each site
kmean = df_results[['site_name','Date','k']].groupby(by=[df_results.Date,'site_name']).mean(numeric_only=True)
kmean = kmean.reset_index(level=1)
kmean.reset_index(inplace=True)
kmean = kmean.rename(columns={'index':'Date','k':'k_bar'})
kmean = pd.merge(df_results,kmean,on=['Date','site_name'],how='left')
kmean.index = df_results.index

df_results['k_bar'] = kmean.k_bar

# create sky condition that cloudy = [0,0.3), partly cloudy = [0.3,0.6), clear = [0.6,)
df_results['sky_condition'] = df_results['k_bar'].apply(lambda x: 'cloudy' if x < 0.3 else ('partly_cloudy' if x < 0.6 else 'clear'))

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_squared_error

fig, axes = plt.subplots(2, 3, figsize=(15, 10), sharey='row')

sky_condition_names = ['Clear sky', 'Partly cloudy sky', 'Cloudy sky']

# Define three different color sets for each subplot
color_sets = [['#86A789', '#D2E3C8'], ['#22668D', '#8ECDDD'], ['#2D3250', '#7077A1']]

for i, condition in enumerate(['clear', 'partly_cloudy', 'cloudy']):
    filter_test_df = df_results[df_results['sky_condition'] == condition]
    mae = filter_test_df.groupby('hour')[['I', 'Ihat']].apply(lambda x: mean_absolute_error(x['I'], x['Ihat'])).reset_index(name='MAE')
    mbe = filter_test_df.groupby('hour')[['I', 'Ihat']].apply(lambda x: x['Ihat'].mean() - x['I'].mean()).reset_index(name='MBE')
    
    # Use different color sets for each subplot
    bar_colors_mae = [color_sets[i][0] if 10 <= hour <= 15 else color_sets[i][1] for hour in mae['hour']]
    bar_colors_mbe = [color_sets[i][0] if 10 <= hour <= 15 else color_sets[i][1] for hour in mbe['hour']]
    
    axes[0, i].bar(mae['hour'], mae['MAE'], color=bar_colors_mae)
    axes[0, i].set_title(f'{sky_condition_names[i]} (n={len(filter_test_df)})', fontsize=20)
    axes[0, i].set_xlabel('Hour', fontsize=20)
    axes[0, i].set_ylabel('MAE [W/sqm]', fontsize=20)
    
    axes[1, i].bar(mbe['hour'], mbe['MBE'], color=bar_colors_mbe)
    axes[1, i].set_xlabel('Hour', fontsize=20)
    axes[1, i].set_ylabel('MBE [W/sqm]', fontsize=20)

fig.tight_layout()
fig.subplots_adjust(top=0.85)
plt.show()


# show the model details
print('\nModel details')
print(model)
print(f'The model has {count_parameters(model):,} trainable parameters')
# overall RMSE, MAE by sky_condition

sky_condition_mae = df_results.groupby('sky_condition')[['I', 'Ihat']].apply(lambda x: mean_absolute_error(x['I'], x['Ihat'])).reset_index(name='MAE')
sky_condition_rmse = df_results.groupby('sky_condition')[['I', 'Ihat']].apply(lambda x: np.sqrt(mean_squared_error(x['I'], x['Ihat']))).reset_index(name='RMSE')

overall_mae = mean_absolute_error(df_results['I'], df_results['Ihat'])
overall_rmse = np.sqrt(mean_squared_error(df_results['I'], df_results['Ihat']))
print(f'Overall MAE: {overall_mae:.2f}')
print(f'Overall RMSE: {overall_rmse:.2f}')

print('\nMAE by sky_condition')
print(sky_condition_mae)

print('\nRMSE by sky_condition')
print(sky_condition_rmse)